# 🧠 RNNs as Computational Dynamical Systems

## Tutorial Introduction

**Welcome!** In this tutorial, we'll explore recurrent neural networks (RNNs) through the powerful lens of dynamical systems theory. Rather than viewing RNNs simply as sequence processors, we'll understand them as dynamical systems that can be analyzed, visualized, and compared to their biological counterparts.

### What You'll Learn

1. **RNNs = Dynamical Systems**: How to formulate RNNs as continuous-time ODEs
2. **The Lorenz System**: Our benchmark chaotic system for testing
3. **Biological Constraints**: E/I balance, Dale's law, spiking neurons
4. **Analysis Tools**: Fixed points, Lyapunov exponents, attractor geometry

### Tutorial Structure

| Notebook | Content | Duration |
|----------|---------|----------|
| **00 (this)** | Introduction, Lorenz system, setup | 30 min |
| **01** | Continuous-Time RNN | 45 min |
| **02** | Balanced E/I Rate Network | 45 min |
| **03** | Balanced Spiking Network | 45 min |
| **04** | Dynamical Systems Analysis | 30 min |
| **05** | Synthesis & Comparison | 30 min |

---
## 🚀 Setup

First, let's install dependencies and set up our environment.

In [ ]:
# Install dependencies (run this cell on Colab)
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q torch torchdiffeq norse matplotlib scipy tqdm
    
    # Clone the tutorial repository
    !git clone -q https://github.com/YOUR_USERNAME/rnn-dynamical-systems-tutorial.git
    %cd rnn-dynamical-systems-tutorial
    
    # Add src to path
    sys.path.insert(0, 'src')

In [ ]:
# Core imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
from scipy.integrate import solve_ivp

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

---
## 📐 Part 1: RNNs as Dynamical Systems

### The Key Insight

A recurrent neural network can be written as a **discrete-time dynamical system**:

$$\mathbf{h}_{t+1} = f(\mathbf{h}_t, \mathbf{x}_t; \theta)$$

Or equivalently, as a **continuous-time dynamical system**:

$$\tau \frac{d\mathbf{h}}{dt} = -\mathbf{h} + \phi(\mathbf{W}_{rec}\mathbf{h} + \mathbf{W}_{in}\mathbf{x} + \mathbf{b})$$

This formulation lets us use powerful tools from dynamical systems theory:

- **Fixed points**: Where $\frac{d\mathbf{h}}{dt} = 0$
- **Stability analysis**: Eigenvalues of the Jacobian
- **Attractors**: Stable patterns the system converges to
- **Lyapunov exponents**: Measure of chaos

### Why This Perspective Matters

1. **Interpretability**: We can understand *what* the network has learned in terms of attractor landscapes
2. **Comparison to biology**: Neural circuits are continuous-time dynamical systems
3. **Universal approximation**: RNNs can approximate any dynamical system (given enough capacity)
4. **Analysis tools**: We can find fixed points, bifurcations, and chaos signatures

---
## 🦋 Part 2: The Lorenz System — Our Benchmark

The [Lorenz system](https://en.wikipedia.org/wiki/Lorenz_system) is a classic chaotic dynamical system that will serve as our ground truth throughout this tutorial. It was discovered by Edward Lorenz in 1963 while studying atmospheric convection.

### The Equations

$$\frac{dx}{dt} = \sigma(y - x)$$
$$\frac{dy}{dt} = x(\rho - z) - y$$
$$\frac{dz}{dt} = xy - \beta z$$

With standard parameters: $\sigma = 10$, $\rho = 28$, $\beta = 8/3$

### Key Properties

- **Chaotic**: Sensitive dependence on initial conditions
- **Strange attractor**: The famous "butterfly" shape
- **Dissipative**: Volume in phase space contracts
- **Low-dimensional**: Only 3 state variables

In [ ]:
def lorenz_system(t, state, sigma=10.0, rho=28.0, beta=8/3):
    """
    Lorenz system ODEs.
    
    Parameters
    ----------
    t : float
        Time (not used, but required by ODE solvers)
    state : array-like
        Current state [x, y, z]
    sigma, rho, beta : float
        System parameters
        
    Returns
    -------
    derivatives : list
        [dx/dt, dy/dt, dz/dt]
    """
    x, y, z = state
    return [
        sigma * (y - x),           # dx/dt
        x * (rho - z) - y,         # dy/dt
        x * y - beta * z           # dz/dt
    ]

In [ ]:
# Generate a Lorenz trajectory
t_span = (0, 100)  # Time range
t_eval = np.linspace(0, 100, 10000)  # Time points to evaluate
initial_state = [1.0, 1.0, 1.0]  # Initial condition

# Integrate
solution = solve_ivp(
    lorenz_system, 
    t_span, 
    initial_state,
    t_eval=t_eval,
    method='RK45',
    rtol=1e-10,
    atol=1e-12
)

# Extract trajectory
t = solution.t
trajectory = solution.y.T  # Shape: (n_times, 3)

print(f"Generated trajectory: {trajectory.shape[0]} time points, {trajectory.shape[1]} dimensions")
print(f"Time step: {t[1] - t[0]:.4f}")

In [ ]:
# Visualize the Lorenz attractor
fig = plt.figure(figsize=(14, 5))

# 3D view
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], 
         lw=0.5, alpha=0.8, color='steelblue')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Lorenz Attractor (3D)')
ax1.view_init(elev=20, azim=45)

# Time series
ax2 = fig.add_subplot(122)
ax2.plot(t[:2000], trajectory[:2000, 0], label='x', alpha=0.8)
ax2.plot(t[:2000], trajectory[:2000, 1], label='y', alpha=0.8)
ax2.plot(t[:2000], trajectory[:2000, 2], label='z', alpha=0.8)
ax2.set_xlabel('Time')
ax2.set_ylabel('Value')
ax2.set_title('Lorenz Time Series')
ax2.legend()

plt.tight_layout()
plt.show()

### 🔍 Explore: Sensitivity to Initial Conditions

A hallmark of chaotic systems is **sensitive dependence on initial conditions** — tiny differences grow exponentially. Let's visualize this.

In [ ]:
# Two trajectories with slightly different initial conditions
ic1 = [1.0, 1.0, 1.0]
ic2 = [1.0 + 1e-8, 1.0, 1.0]  # Tiny perturbation!

sol1 = solve_ivp(lorenz_system, (0, 50), ic1, t_eval=np.linspace(0, 50, 5000))
sol2 = solve_ivp(lorenz_system, (0, 50), ic2, t_eval=np.linspace(0, 50, 5000))

# Compute distance between trajectories
distance = np.sqrt(np.sum((sol1.y - sol2.y)**2, axis=0))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trajectories
axes[0].plot(sol1.t, sol1.y[0], 'b-', label='Trajectory 1', alpha=0.8)
axes[0].plot(sol2.t, sol2.y[0], 'r--', label='Trajectory 2 (perturbed by 1e-8)', alpha=0.8)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('x')
axes[0].set_title('Two Nearly Identical Initial Conditions')
axes[0].legend()

# Distance (log scale)
axes[1].semilogy(sol1.t, distance)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Distance (log scale)')
axes[1].set_title('Exponential Divergence')
axes[1].axhline(1e-8, color='gray', ls='--', label='Initial separation')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Initial separation: {1e-8:.0e}")
print(f"Final separation: {distance[-1]:.2f}")
print(f"Amplification factor: {distance[-1]/1e-8:.2e}")

### 🎯 Our Task: Lorenz Trajectory Prediction

**Goal**: Train an RNN to predict the next state of the Lorenz system given its current state.

$$\hat{\mathbf{s}}_{t+\Delta t} = \text{RNN}(\mathbf{s}_t)$$

where $\mathbf{s} = [x, y, z]^T$.

This is interesting because:
1. The dynamics are highly nonlinear
2. Long-term prediction is fundamentally limited by chaos
3. We can analyze what the trained RNN has learned

---
## 📊 Part 3: Preparing the Data

Let's create training, validation, and test sets from our Lorenz trajectory.

In [ ]:
# Generate a longer trajectory for training
def generate_lorenz_data(total_time=200, dt=0.01, transient=10, seed=42):
    """
    Generate Lorenz trajectory data.
    
    Parameters
    ----------
    total_time : float
        Total simulation time
    dt : float
        Time step
    transient : float
        Time to discard (to reach attractor)
    seed : int
        Random seed
        
    Returns
    -------
    t : np.ndarray
        Time points
    trajectory : np.ndarray
        States, shape (n_times, 3)
    """
    np.random.seed(seed)
    
    # Random initial condition near attractor
    ic = [1.0 + np.random.randn()*0.1, 
          1.0 + np.random.randn()*0.1, 
          1.0 + np.random.randn()*0.1]
    
    t_eval = np.arange(0, total_time + transient, dt)
    
    sol = solve_ivp(
        lorenz_system,
        (0, total_time + transient),
        ic,
        t_eval=t_eval,
        method='RK45',
        rtol=1e-10
    )
    
    # Remove transient
    transient_steps = int(transient / dt)
    t = sol.t[transient_steps:] - transient
    trajectory = sol.y[:, transient_steps:].T
    
    return t, trajectory

# Generate data
t_data, trajectory_data = generate_lorenz_data(total_time=200, dt=0.01)
print(f"Generated {len(t_data)} time steps ({t_data[-1]:.0f} time units)")

In [ ]:
# Split into train/val/test
n_total = len(trajectory_data)
n_train = int(0.7 * n_total)
n_val = int(0.15 * n_total)
n_test = n_total - n_train - n_val

train_data = trajectory_data[:n_train]
val_data = trajectory_data[n_train:n_train+n_val]
test_data = trajectory_data[n_train+n_val:]

print(f"Train: {len(train_data)} steps ({len(train_data)*0.01:.0f} time units)")
print(f"Val: {len(val_data)} steps ({len(val_data)*0.01:.0f} time units)")
print(f"Test: {len(test_data)} steps ({len(test_data)*0.01:.0f} time units)")

In [ ]:
# Normalize data (important for training!)
train_mean = train_data.mean(axis=0)
train_std = train_data.std(axis=0)

train_normalized = (train_data - train_mean) / train_std
val_normalized = (val_data - train_mean) / train_std
test_normalized = (test_data - train_mean) / train_std

print(f"Normalization stats:")
print(f"  Mean: {train_mean}")
print(f"  Std: {train_std}")

In [ ]:
# Create PyTorch datasets
from torch.utils.data import Dataset, DataLoader

class LorenzDataset(Dataset):
    """
    Dataset for Lorenz prediction task.
    
    Given a sequence of states, predict the next state.
    """
    def __init__(self, trajectory, seq_length=50):
        """
        Parameters
        ----------
        trajectory : np.ndarray
            Shape (n_times, 3)
        seq_length : int
            Input sequence length
        """
        self.trajectory = torch.tensor(trajectory, dtype=torch.float32)
        self.seq_length = seq_length
        
    def __len__(self):
        return len(self.trajectory) - self.seq_length
    
    def __getitem__(self, idx):
        # Input: sequence of states
        x = self.trajectory[idx:idx+self.seq_length]
        # Target: next state
        y = self.trajectory[idx+self.seq_length]
        return x, y

# Create datasets
seq_length = 50  # Use 50 time steps as input

train_dataset = LorenzDataset(train_normalized, seq_length)
val_dataset = LorenzDataset(val_normalized, seq_length)
test_dataset = LorenzDataset(test_normalized, seq_length)

# Create dataloaders
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\nDataset sizes:")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Val: {len(val_dataset)} samples")
print(f"  Test: {len(test_dataset)} samples")

In [ ]:
# Visualize a sample
sample_x, sample_y = train_dataset[0]
print(f"Input shape: {sample_x.shape}")
print(f"Target shape: {sample_y.shape}")

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for i, (ax, dim) in enumerate(zip(axes, ['x', 'y', 'z'])):
    ax.plot(range(seq_length), sample_x[:, i].numpy(), 'b-', label='Input')
    ax.scatter([seq_length], [sample_y[i].numpy()], c='red', s=100, zorder=5, label='Target')
    ax.set_xlabel('Time step')
    ax.set_ylabel(dim)
    ax.legend()
    ax.set_title(f'Dimension: {dim}')

plt.suptitle('Sample: Predict next state from sequence', y=1.02)
plt.tight_layout()
plt.show()

---
## 🧪 Part 4: Preview — The Networks We'll Build

Over the next notebooks, we'll implement and compare three network architectures:

### 1. Continuous-Time RNN (CT-RNN)
$$\tau \frac{d\mathbf{h}}{dt} = -\mathbf{h} + \tanh(\mathbf{W}_{rec}\mathbf{h} + \mathbf{W}_{in}\mathbf{x} + \mathbf{b})$$

- Smooth dynamics
- Amenable to ODE analysis
- Trained using Neural ODEs (adjoint method)

### 2. Balanced E/I Rate Network
$$\tau_E \frac{d\mathbf{r}_E}{dt} = -\mathbf{r}_E + \phi(\mathbf{W}_{EE}\mathbf{r}_E - \mathbf{W}_{EI}\mathbf{r}_I + \mathbf{I}_{ext})$$
$$\tau_I \frac{d\mathbf{r}_I}{dt} = -\mathbf{r}_I + \phi(\mathbf{W}_{IE}\mathbf{r}_E - \mathbf{W}_{II}\mathbf{r}_I)$$

- Separate excitatory (E) and inhibitory (I) populations
- Dale's law: E neurons have positive weights, I neurons have negative
- Dynamically balanced regime

### 3. Balanced Spiking Network
$$\tau_m \frac{dV}{dt} = -(V - V_{rest}) + I_{syn} + I_{ext}$$
$$\text{if } V > V_{th}: \text{spike}, V \rightarrow V_{reset}$$

- Leaky Integrate-and-Fire (LIF) neurons
- Event-driven communication (spikes)
- Reservoir computing approach

---
## 📝 Summary

In this introduction, we've:

1. ✅ Set up our environment and imports
2. ✅ Understood RNNs as dynamical systems
3. ✅ Explored the Lorenz system — our benchmark
4. ✅ Prepared training, validation, and test data
5. ✅ Previewed the three network architectures

### Next: Notebook 01 — Continuous-Time RNN

We'll implement a CT-RNN using Neural ODEs and train it on the Lorenz prediction task.

---
## 💾 Save Data for Later Notebooks

Let's save the preprocessed data so we can use it across notebooks.

In [ ]:
# Save data
import os
os.makedirs('data/processed', exist_ok=True)

np.savez(
    'data/processed/lorenz_data.npz',
    train=train_normalized,
    val=val_normalized,
    test=test_normalized,
    mean=train_mean,
    std=train_std,
    dt=0.01,
    seq_length=seq_length
)

print("Data saved to data/processed/lorenz_data.npz")

In [ ]:
# Utility function to load data in other notebooks
def load_lorenz_data(path='data/processed/lorenz_data.npz'):
    """
    Load preprocessed Lorenz data.
    
    Returns
    -------
    data : dict
        Contains train, val, test arrays and normalization params
    """
    data = np.load(path)
    return {
        'train': data['train'],
        'val': data['val'],
        'test': data['test'],
        'mean': data['mean'],
        'std': data['std'],
        'dt': float(data['dt']),
        'seq_length': int(data['seq_length'])
    }

# Test loading
loaded = load_lorenz_data()
print(f"Loaded data shapes: train={loaded['train'].shape}, val={loaded['val'].shape}, test={loaded['test'].shape}")